In [13]:
import pandas as pd
import numpy as np
import re

pd.set_option("display.max_rows", 1538)

In [6]:
df = pd.read_csv("data/GSAF5.csv", encoding = "ISO-8859-1")

In [7]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [15]:
df["Species "].value_counts().sort_values(ascending=False)

                                                                                                                                                                                      1
Remains recovered 5 days later                                                                                                                                                                            1
Lemon shark, 2 m [6'9"]                                                                                                                                                                                   1
1.2 m [4'] shark (spinner shark?)                                                                                                                                                                         1
Small dusky shark or blackfin shark                                                                                                                                                                       1


In [4]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
df.shape

(5992, 24)

# Clean `Date` column

In [6]:
# I'm creating a function that can be applied to an entire column of a DataFrame. In this case, it is necessary to 'catch' the 
# exception (error) so that the function runs, as errors breaks the code. The function intentionally return a boolean so that
# it can be used for subestting DataFrames.
# You can catch exceptions using a so-called 'try except block' as shown here:

def test_to_datetime(ts):
    try:
        pd.to_datetime(ts, infer_datetime_format=True)
    except ValueError:
        return False
    return True
        

In [7]:
np.isnat(np.datetime64('NaT'))

True

In [8]:
test_to_datetime("16-Sep-16")

True

In [9]:
# We can now use our function on every record in a column using the .apply() method on a Series like so:
df[df["Date"].apply(test_to_datetime) == False].shape

(857, 24)

In [10]:
df[df["Date"].apply(test_to_datetime) == False].head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
33,2016.07.14.4,Reported 14-Jul-2016,2016,Unprovoked,BAHAMAS,NaN,Tiger Beach,Scuba Diving,Michael Dornellas,M,...,"Lemon shark, 9'","GrindTV, 7/14/2016",2016.07.14.R-TigerBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.14.R,2016.07.14.4,5960,NaN,NaN
34,2016.07.08.R,Reported 08-Jul-2016,2016,Unprovoked,SPAIN,Canary Islands,"Las Teresitas, Tenerife",Wading,female,F,...,Angel shark,La Opinion de Tenerife,2016.07.08.R-Spain.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.08.R,2016.07.08.R,5959,NaN,NaN
85,2016.03.03.R,Reported 03-Mar-2016,2016,Unprovoked,AUSTRALIA,South Australia,Wrights Bay,Fishing,Lee Taplin,M,...,Bronze whaler,"9 News, 3/1/2016",2016.03.03.R-Taplin.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.03.03.R,2016.03.03.R,5908,NaN,NaN
90,2016.02.10.R,Reported 10-Feb-2016,2016,Invalid,CAYMAN ISLANDS,Grand Cayman,Stingray City Bar,Feeding stingrays?,Richard Branson,M,...,No shark involvement,R. Branson,2016.02.10.R-Branson-stingray.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.02.10.R,2016.02.10.R,5903,NaN,NaN
100,2016.01.11.R,Reported 11-Jan-2016,2016,Unprovoked,AUSTRALIA,Queensland,"Happy Valley Beach, Caloundra",Surfing,Shane Hilder,M,...,Wobbegong shark,"ABC Sunshine Coast, 1/11/2016",2016.01.11.R-Hilder.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.01.11.R,2016.01.11.R,5893,NaN,NaN


See if we can infer date from "Case Number" field

Looks like `Case Number` could be used to get the date

In [11]:
(df["Case Number"]
 .str.replace("-", ".")
 .str.replace(",", ".")
 .str.extract(r"(^\d{4}\.\d{2}.\d{2})", expand=False)
 .apply(test_to_datetime)
 .value_counts())

True     5256
False     736
Name: Case Number, dtype: int64

We can infer another 5256 records using the `pd.to_datetime()` function when using `Case number` and adding a few manipulations!

In [12]:
df_ncasen = (df[df["Case Number"]
                .str.replace("-", ".")
                .str.replace(",", ".")
                .str.extract(r"(^\d{4}\.\d{2}.\d{2})", expand=False)
                .apply(test_to_datetime) == False])

In [13]:
df_ncasen_nzero = df_ncasen[(df_ncasen["Case Number"]
                             .str.extract(r"(^\d{4}\.\d{2}.\d{2})", expand=False)
                             .str.replace(r"(00$)", "01")
                             .str.replace(r"\.00\.", ".01.")).apply(test_to_datetime) == False]

In [14]:
df_ncasen_nzero.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
5855,1642.00.00.b,Late 1600s Reported 1728,1642,Invalid,GUINEA,NaN,NaN,Went overboard,crew member of the Nieuwstadt,M,...,NaN,"History of the Pyrates, by D. Defoe, Vol. 2, p.28",1642.00.00.b-Nieuwstadt.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1642.00.00.b,1642.00.00.b,138,NaN,NaN
5856,1638.00.00.R,Reported 1638,1638,Unprovoked,NaN,NaN,NaN,NaN,sailors,M,...,NaN,Sir Thomas Herbert,1638.00.00.R-Herbert,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1638.00.00.R,1638.00.00.R,137,NaN,NaN
5857,1637.00.00.R,Reported 1637,1637,Unprovoked,INDIA,West Bengal,Hooghly River mouth,Wading,Hindu pilgrims,NaN,...,NaN,"H. Edwards, p.31, citing Sebastian Manrique",1637.00.00.R-Manrique.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1637.00.00.R,1637.00.00.R,136,NaN,NaN
5858,1617.00.00.R,Reported 1617,1617,Unprovoked,INDIA,West Bengal,Ganges Delta,NaN,Indian people,NaN,...,NaN,"H. Edwards, p.31, citing Samuel Purchas",1617.00.00-Purchas.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1617.00.00.R,1617.00.00.R,135,NaN,NaN
5859,1642.00.00,1642,1642,Unprovoked,USA,New York,Between Manhattan and The Bronx,Swimming,Antony Van Corlear,M,...,NaN,"Knickerbocker's History of New York, by Washin...",1642.00.00-Corlear.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1642.00.00,1642.00.00,134,NaN,NaN


For the remaining ones, we do not care about dates before 1642.
Let's go through each steps to create a clean date column

In [15]:
df.assign(new_date=np.nan)
df["new_date"] = pd.to_datetime(df[df["Date"].apply(test_to_datetime)]["Date"])

In [16]:
def infer_from_case_number(case_number):
    try:
        return pd.to_datetime(case_number, infer_datetime_format=True)
    except:
        return np.nan

In [17]:
df.loc[df["Date"].apply(test_to_datetime) == False, "new_date"] = (df[df["Date"].apply(test_to_datetime) == False]["Case Number"]
                                                                   .str.replace("-", ".")
                                                                   .str.replace(",", ".")
                                                                   .str.extract(r"(^\d{4}\.\d{2}.\d{2})", expand=False)
                                                                   .str.replace(r"(00$)", "01")
                                                                   .str.replace(r"\.00\.", ".01.")).apply(infer_from_case_number)

In [18]:
df["new_date"].isnull().value_counts()

False    5855
True      137
Name: new_date, dtype: int64

In [19]:
df[df["new_date"].isnull()].head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,new_date
5855,1642.00.00.b,Late 1600s Reported 1728,1642,Invalid,GUINEA,NaN,NaN,Went overboard,crew member of the Nieuwstadt,M,...,"History of the Pyrates, by D. Defoe, Vol. 2, p.28",1642.00.00.b-Nieuwstadt.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1642.00.00.b,1642.00.00.b,138,NaN,NaN,NaT
5856,1638.00.00.R,Reported 1638,1638,Unprovoked,NaN,NaN,NaN,NaN,sailors,M,...,Sir Thomas Herbert,1638.00.00.R-Herbert,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1638.00.00.R,1638.00.00.R,137,NaN,NaN,NaT
5857,1637.00.00.R,Reported 1637,1637,Unprovoked,INDIA,West Bengal,Hooghly River mouth,Wading,Hindu pilgrims,NaN,...,"H. Edwards, p.31, citing Sebastian Manrique",1637.00.00.R-Manrique.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1637.00.00.R,1637.00.00.R,136,NaN,NaN,NaT
5858,1617.00.00.R,Reported 1617,1617,Unprovoked,INDIA,West Bengal,Ganges Delta,NaN,Indian people,NaN,...,"H. Edwards, p.31, citing Samuel Purchas",1617.00.00-Purchas.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1617.00.00.R,1617.00.00.R,135,NaN,NaN,NaT
5859,1642.00.00,1642,1642,Unprovoked,USA,New York,Between Manhattan and The Bronx,Swimming,Antony Van Corlear,M,...,"Knickerbocker's History of New York, by Washin...",1642.00.00-Corlear.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1642.00.00,1642.00.00,134,NaN,NaN,NaT


Extract the first year if it finds one in the Date column

In [20]:
def get_first_element(l):
    return next(iter(l), np.nan)

In [21]:
def safe_to_datetime(date):
    try:
        return pd.to_datetime(date, infer_datetime_format=True)
    except:
        return np.nan

In [22]:
df.loc[df["new_date"].isnull(), "new_date"] = (df[df["new_date"].isnull()]["Date"]
                                             .str.findall(r"\d{4}")
                                             .apply(get_first_element)
                                             .astype(str)
                                             .apply(safe_to_datetime))

In [23]:
df = df.drop(df[df["new_date"].isnull()].index)

# Inspect the `Type` column

In [24]:
df["Type"].value_counts()

Unprovoked      4365
Provoked         557
Invalid          518
Sea Disaster     216
Boat             200
Boating          110
Name: Type, dtype: int64

In [25]:
df.loc[df["Type"] == "Boating", "Type"] = "Boat"

In [26]:
df["Type"].value_counts()

Unprovoked      4365
Provoked         557
Invalid          518
Boat             310
Sea Disaster     216
Name: Type, dtype: int64

# `Country`

In [27]:
df["Country"].value_counts().shape

(202,)

In [28]:
df["Country"].value_counts()

USA                               2114
AUSTRALIA                         1276
SOUTH AFRICA                       563
PAPUA NEW GUINEA                   131
NEW ZEALAND                        125
BRAZIL                             102
BAHAMAS                             98
MEXICO                              80
ITALY                               70
FIJI                                62
PHILIPPINES                         59
REUNION                             56
NEW CALEDONIA                       51
MOZAMBIQUE                          44
CUBA                                42
SPAIN                               40
EGYPT                               36
CROATIA                             34
INDIA                               34
PANAMA                              32
JAPAN                               32
IRAN                                29
SOLOMON ISLANDS                     29
HONG KONG                           24
JAMAICA                             23
GREECE                   

In [29]:
df["country"] = (df["Country"]
                 .str.upper()
                 .str.strip()
                 .str.replace("?",""))